In [1]:
%run proof_setup

## Real case

First, the real case is relatively simple.
The integral that we want to do is:

$$
k_\Delta(\tau) = \int_{t_i-\Delta/2}^{t_i+\Delta/2} \mathrm{d}t \,\int_{t_j-\Delta/2}^{t_j+\Delta/2}\mathrm{d}t^\prime\,k(t - t^\prime)
$$

For celerite kernels it helps to make the assumtion that $t_j + \Delta/2 < t_i - \Delta/2$ (in other words, the exposure times do not overlap).

In [2]:
import sympy as sm
cr = sm.symbols("cr", positive=True)
ti, tj, dt, t, tp = sm.symbols("ti, tj, dt, t, tp", real=True)

k = sm.exp(-cr*(t - tp))
k0 = k.subs([(t, ti), (tp, tj)])
kint = sm.simplify(sm.integrate(
    sm.integrate(k, (t, ti-dt/2, ti+dt/2)),
    (tp, tj-dt/2, tj+dt/2)) / dt**2)
res = sm.simplify(kint / k0)
print(res)

(exp(2*cr*dt) - 2*exp(cr*dt) + 1)*exp(-cr*dt)/(cr**2*dt**2)


This is the factor that we want.
Let's make sure that it is identical to what we have in the note.

In [3]:
kD = 2 * (sm.cosh(cr*dt) - 1) / (cr*dt)**2
sm.simplify(res.expand() - kD.expand())

0

Excellent.

## Complex case

The complex cases proceeds similarly, but it's a bit more involved.
In this case,

$$
k(\tau) = (a + i\,b)\,\exp(-(c+i\,d)\,(t_i-t_j))
$$

In [4]:
a, b, c, d, tpp = sm.symbols("a, b, c, d, tpp", real=True)
k = sm.exp(-(c + sm.I*d) * tp)
kint = sm.simplify(sm.integrate(k, (tp, ti-dt/2, ti+dt/2)) *
                   sm.integrate(k.subs([(tp, -tpp)]),
                                (tpp, tj-dt/2, tj+dt/2)) / dt**2)

res = sm.simplify(kint)
print(res)

(c*exp(2*dt*(c + I*d)) - 2*c*exp(dt*(c + I*d)) + c + I*d*exp(2*dt*(c + I*d)) - 2*I*d*exp(dt*(c + I*d)) + I*d)*exp(-c*dt - c*ti + c*tj - I*d*dt - I*d*ti + I*d*tj)/(dt**2*(c**3 + 3*I*c**2*d - 3*c*d**2 - I*d**3))


That doesn't look so bad!

But, I'm going to re-write it by hand and make sure that it's correct:

In [5]:
coeff = (c-sm.I*d)**2 / (dt*(c**2+d**2))**2
coeff *= (sm.exp((c+sm.I*d)*dt) + sm.exp(-(c+sm.I*d)*dt)-2)
sm.simplify(coeff * sm.exp(-(c+sm.I*d)*(ti-tj)) - kint)

0

Good.

Now we need to work out nice expressions for the real and imaginary parts of this.
First, the real part.
I found that it was easiest to look at the prefactors for the trig functions directly and simplify those.
Here we go:

In [6]:
res = (a+sm.I*b) * coeff
A = sm.simplify((res.expand(complex=True) +
                 sm.conjugate(res).expand(complex=True)) / 2)

In [7]:
sm.simplify(sm.poly(A, sm.cos(dt*d)).coeff_monomial(sm.cos(dt*d)))

(a*c**2*exp(2*c*dt) + a*c**2 - a*d**2*exp(2*c*dt) - a*d**2 + 2*b*c*d*exp(2*c*dt) + 2*b*c*d)*exp(-c*dt)/(dt**2*(c**4 + 2*c**2*d**2 + d**4))

In [8]:
sm.simplify(sm.poly(sm.poly(A, sm.cos(dt*d)).coeff_monomial(1), sm.sin(dt*d)).coeff_monomial(sm.sin(dt*d)))

(2*a*c*d*exp(2*c*dt) - 2*a*c*d - b*c**2*exp(2*c*dt) + b*c**2 + b*d**2*exp(2*c*dt) - b*d**2)*exp(-c*dt)/(dt**2*(c**4 + 2*c**2*d**2 + d**4))

In [9]:
sm.simplify(sm.poly(sm.poly(A, sm.cos(dt*d)).coeff_monomial(1), sm.sin(dt*d)).coeff_monomial(1))

2*(-a*c**2 + a*d**2 - 2*b*c*d)/(dt**2*(c**4 + 2*c**2*d**2 + d**4))

Then, same thing for the imaginary part:

In [10]:
B = sm.simplify(-sm.I * (res.expand(complex=True) -
                         sm.conjugate(res).expand(complex=True)) / 2)

In [11]:
sm.simplify(sm.poly(B, sm.cos(dt*d)).coeff_monomial(sm.cos(dt*d)))

(-2*a*c*d*exp(2*c*dt) - 2*a*c*d + b*c**2*exp(2*c*dt) + b*c**2 - b*d**2*exp(2*c*dt) - b*d**2)*exp(-c*dt)/(dt**2*(c**4 + 2*c**2*d**2 + d**4))

In [12]:
sm.simplify(sm.poly(sm.poly(B, sm.cos(dt*d)).coeff_monomial(1), sm.sin(dt*d)).coeff_monomial(sm.sin(dt*d)))

(a*c**2*exp(2*c*dt) - a*c**2 - a*d**2*exp(2*c*dt) + a*d**2 + 2*b*c*d*exp(2*c*dt) - 2*b*c*d)*exp(-c*dt)/(dt**2*(c**4 + 2*c**2*d**2 + d**4))

In [13]:
sm.simplify(sm.poly(sm.poly(B, sm.cos(dt*d)).coeff_monomial(1), sm.sin(dt*d)).coeff_monomial(1))

2*(2*a*c*d - b*c**2 + b*d**2)/(dt**2*(c**4 + 2*c**2*d**2 + d**4))

Ok.

Now let's make sure that the simplified expressions are right.

In [14]:
C1 = (a*c**2 - a*d**2 + 2*b*c*d)
C2 = (b*c**2 - b*d**2 - 2*a*c*d)
cos_term = (sm.exp(c*dt) + sm.exp(-c*dt)) * sm.cos(d*dt) - 2
sin_term = (sm.exp(c*dt) - sm.exp(-c*dt)) * sm.sin(d*dt)
denom = dt**2 * (c**2 + d**2)**2

A0 = (C1 * cos_term - C2 * sin_term) / denom
B0 = (C2 * cos_term + C1 * sin_term) / denom

In [15]:
sm.simplify(A.expand() - A0.expand())

0

In [16]:
sm.simplify(B.expand() - B0.expand())

0

Finally let's rewrite things in terms of hyperbolic trig functions.

In [17]:
sm.simplify(2*(sm.cosh(c*dt) * sm.cos(d*dt) - 1).expand() - cos_term.expand())

0

In [18]:
sm.simplify(2*(sm.sinh(c*dt) * sm.sin(d*dt)).expand() - sin_term.expand())

0

Looks good!